In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flipkart-fasion-products-dataset/output.xlsx
/kaggle/input/flipkart-fasion-products-dataset/flipkart_fashion_products_dataset.json
/kaggle/input/ecommerce/ecommerce_dataset.csv


In [2]:
df = pd.read_csv('/kaggle/input/ecommerce/ecommerce_dataset.csv',names=['category','description'],header=None)
print(df.shape)
df.head()

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [3]:
df.category.value_counts()

category
Household                 19313
Books                     11820
Electronics               10621
Clothing & Accessories     8671
Name: count, dtype: int64

In [4]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [5]:
df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)

/tmp/ipykernel_13/2912538940.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)


In [6]:

df.category.unique()

array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

In [7]:
df['category'] = '__label__' + df['category'].astype(str)
df.head(5)

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [8]:
df['category_description'] = df['category'] + ' ' + df['description']
df.head(3)

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...


**Preprocessing**

1.Remove punctuation
2.Remove extra space
3.Make the entire sentence lower case

In [9]:
import re

In [10]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower() 

In [11]:
df['category_description'] = df['category_description'].map(preprocess)
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


In [12]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [13]:
train.shape, test.shape

((40339, 3), (10085, 3))

In [14]:
train.to_csv("ecommerce.train", columns=["category_description"], index=False, header=False)
test.to_csv("ecommerce.test", columns=["category_description"], index=False, header=False)

In [15]:

import fasttext

model = fasttext.train_supervised(input="ecommerce.train")
model.test("ecommerce.test")

Read 4M words
Number of words:  79296
Number of labels: 4
Progress: 100.0% words/sec/thread: 1951840 lr:  0.000000 avg.loss:  0.193129 ETA:   0h 0m 0s


(10083, 0.9685609441634434, 0.9685609441634434)

In [16]:
model.predict('wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3')

(('__label__electronics',), array([0.98535192]))

In [17]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")


(('__label__clothing_accessories',), array([1.00001001]))

In [18]:
model.predict('have you written that on the page')

(('__label__books',), array([0.99890381]))

In [19]:
model.predict('how many boys in red shirt uses mobile phone ?!!' )

(('__label__clothing_accessories',), array([0.97376454]))

In [20]:
model.get_nearest_neighbors('shirts')

[(0.9930432438850403, 'punisher'),
 (0.9925774931907654, "buyer's"),
 (0.9924696683883667, 'jalandhar'),
 (0.9918646812438965, "'made"),
 (0.9917641282081604, 'jeging'),
 (0.991267204284668, 'epobs'),
 (0.9909648299217224, 'milancho'),
 (0.9909647107124329, 'colour_free'),
 (0.990689218044281, 'urself'),
 (0.990689218044281, 'outletbrand')]